# Aula 4: SQL

## Introdução

Nesta aula veremos as principais queries de consulta em sql

## Conexao no db4free via sqlalchemy

In [1]:
import yaml
import sqlalchemy
import pandas as pd
from pandasql import sqldf

In [2]:
# Load SQL credentials
with open('credentials.yml', 'r') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

- A engine deve estar no formato:
- mysql://`user`:`password`@`host`:`port`/`database`

In [3]:
engine = sqlalchemy.create_engine(f'mysql://{credentials["user"]}:{credentials["password"]}@{credentials["host"]}:{credentials["port"]}/{credentials["database"]}') # connect to server

## GROUP BY

- Podemos agrupar a tabela por uma chave e sumarizar os valores 
- Por exemplo, podemos analisar o número de cada `product_category` 
- `GROUP BY` sempre está acompanhando de alguma função de agregação: `COUNT()`, `SUM()`, `AVG()`

In [6]:
import os

In [7]:
os.listdir()

['.ipynb_checkpoints',
 'AtrasoCartaoCredito.csv',
 'Aula 01 - SQL - Códigos.ipynb',
 'Aula 01 - SQL.ipynb',
 'Aula 02 - SQL - Códigos.ipynb',
 'Aula 02 - SQL - Exercícios.ipynb',
 'Aula 02 - SQL.ipynb',
 'Aula 03 - SQL - Códigos.ipynb',
 'Aula 03 - SQL - Exercícios (Gabarito).ipynb',
 'Aula 03 - SQL - Exercícios.ipynb',
 'Aula 03 - SQL.ipynb',
 'Aula 04 - SQL - Códigos.ipynb',
 'Aula 04 - SQL - Exercícios (Gabarito).ipynb',
 'Aula 04 - SQL - Exercícios.ipynb',
 'Aula 04 - SQL.ipynb',
 'Aula 05 - SQL TODO subquery.ipynb',
 'credentials.yml',
 'DS.JPG',
 'Exercicios SQL - Semana 01.docx',
 'Plano de aula - SQL.docx',
 'salesOlist.db']

In [4]:
AtrasoCartaoCredito = pd.read_csv('AtrasoCartaoCredito.csv')

In [5]:
AtrasoCartaoCredito.head()

,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282
3,4,1,1,mulher,35,1,531.0,37.928571,44.250000,PR,Congonhinhas,Região Sul,2.528571
4,5,0,3,homem,35,0,805.0,57.500000,67.083333,BA,Morpará,Região Nordeste,3.833333


In [13]:
AtrasoCartaoCredito['GrupoEconomico'].value_counts().sort_index()

1    216
2    184
3    491
Name: GrupoEconomico, dtype: int64

In [11]:
query = '''
    SELECT GrupoEconomico, COUNT(*) AS num_grupo_economico
    FROM AtrasoCartaoCredito
    GROUP BY GrupoEconomico
'''

sqldf(query)

,GrupoEconomico,num_grupo_economico
0,1,216
1,2,184
2,3,491


## ORDER BY

Podemos ordernar a consulta por alguma coluna com `ORDER BY` em ordem crescente

In [14]:
query = '''
    SELECT GrupoEconomico, COUNT(*) AS num_grupo_economico
    FROM AtrasoCartaoCredito
    GROUP BY GrupoEconomico
    ORDER BY GrupoEconomico
'''

sqldf(query)

,GrupoEconomico,num_grupo_economico
0,1,216
1,2,184
2,3,491


- Ou em ordem descrescente
- Também podemos renomear a coluna com o alias `AS`

In [16]:
AtrasoCartaoCredito.head(3)

,ID,Target,GrupoEconomico,Sexo,Idade,GrupoRisco,ValorCompraAnual,GastoMax,GastoMedio,UF,CidadeResidencia,RegiaodoPais,NumeroComprasOnline
0,1,0,3,homem,22,1,7250.0,517.857143,604.166667,MG,Morada Nova de Minas,Região Sudeste,54.924242
1,2,1,1,mulher,38,1,71283.0,5091.642857,5940.250000,RN,Marcelino Vieira,Região Nordeste,312.644737
2,3,1,3,mulher,26,0,7925.0,566.071429,660.416667,PR,Agudos do Sul,Região Sul,50.801282


In [19]:
query = '''
    SELECT GrupoEconomico, Sexo, COUNT(*) AS freq
    FROM AtrasoCartaoCredito
    GROUP BY GrupoEconomico, Sexo
    ORDER BY GrupoEconomico DESC
'''

sqldf(query)

,GrupoEconomico,Sexo,freq
0,3,homem,347
1,3,mulher,144
2,2,homem,108
3,2,mulher,76
4,1,homem,122
5,1,mulher,94


## HAVING

Quando queremos aplicar um filtro após fazer o `GROUP BY`, usamos o comando `HAVING`

In [22]:
query = '''
    SELECT GrupoEconomico, Sexo, COUNT(*) AS freq
    FROM AtrasoCartaoCredito
    GROUP BY GrupoEconomico, Sexo
'''

sqldf(query)

,GrupoEconomico,Sexo,freq
0,1,homem,122
1,1,mulher,94
2,2,homem,108
3,2,mulher,76
4,3,homem,347
5,3,mulher,144


In [25]:
query = '''
    SELECT GrupoEconomico, Sexo, COUNT(*) AS freq
    FROM AtrasoCartaoCredito
    WHERE GrupoEconomico <> 1
    GROUP BY GrupoEconomico, Sexo
    HAVING COUNT(*) > 100
'''

sqldf(query)

,GrupoEconomico,Sexo,freq
0,2,homem,108
1,3,homem,347
2,3,mulher,144


## Pausa - voltamos 20:37